## Steps
 * get 10000 links
 * get data from links
 * see where all required data is saved
 * clean data (empty rows, numerical values, etc.)


In [2]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import os

date = strftime("%Y-%m-%d")

import selenium

# The selenium.webdriver module provides all the implementations of WebDriver
# Currently supported are Firefox, Chrome, IE and Remote. The `Keys` class provides keys on
# the keyboard such as RETURN, F1, ALT etc.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

## Get the initial link and its data

Using the page in the url we loop through the pages untill we have as many links as we need (currently 100). This takes a while, due to needing time to fully load the page before we can get the data (30 links per page though, maybe something I can do with asynchronous stuff so we don't have to wait as long(just rand time between 0 and 2 sec maybe))

In [3]:
links = []

page = 1

while len(links) < 100:
    url = f"https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page={page}&orderBy=newest"

    driver = webdriver.Firefox()
    driver.implicitly_wait(5)
    driver.get(url)

    #python_button = driver.find_elements_by_xpath('//div[@data-reactid="269"]')[0]
    #python_button.click()

    # And then we use Beautiful soup
    soup = BeautifulSoup(driver.page_source)

    driver.close()

    for list_item in soup.find_all("li", attrs={"class": "search-results__item"}):
        for link in list_item.find_all("a", attrs={"class": "card__title-link"}):
            links.append(link.get("href"))
    
    page += 1

## Print the links

Just to get a bit of an overview, can we maybe spot a pattern? (Sadly, no)

In [4]:
for link in links:
    print(link)

https://www.immoweb.be/en/classified/apartment-block/for-sale/evere/1140/9777739?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/apartment/for-sale/antwerpen/2000/9777734?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/house/for-sale/duffel/2570/9777733?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/house/for-sale/berchem/2018/9777732?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/house/for-sale/braine-lalleud/1420/9777730?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/villa/for-sale/linkebeek/1630/9777724?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/apartment/for-sale/forest/1190/9777723?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/apartment/for-sale/heverlee/3001/9777722?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/apartment-block/for-sale/liege/4020/9777720?searchId=621cad6b3ef52
https://www.immoweb.be/en/classified/apartment/for-sale/antwerp/2600/9777719?searchId=621cad6b3ef5

## Get the data from one of the links

We now make a soup from just 1 of the links so we can start looking at the data and which data we need. Later we will replace this with a loop so we can start building soups from the different links. Currently using just a regular get request, hopefully we can get all the necessary data with this, not really looking forward to having to start and close drivers for every link.

In [5]:
r = requests.get(links[0])
print(links[0], r.status_code)
soup = BeautifulSoup(r.content, "lxml")

https://www.immoweb.be/en/classified/apartment-block/for-sale/evere/1140/9777739?searchId=621cad6b3ef52 200


In [6]:
script_text = soup.find_all("script")
script_text = soup.find_all("script")[1].text.split("= ", 1)[1]

json_data = json.loads(script_text[:script_text.find(";")])
print(json_data[0]['classified'])

{'id': '9777739', 'type': 'house', 'subtype': 'apartment block', 'price': '650000', 'transactionType': 'for sale', 'zip': '1140', 'visualisationOption': 'm', 'kitchen': {'type': 'installed'}, 'building': {'constructionYear': '1938', 'condition': 'good'}, 'energy': {'heatingType': 'gas'}, 'certificates': {'primaryEnergyConsumptionLevel': ''}, 'bedroom': {'count': '4'}, 'land': {'surface': '130'}, 'atticExists': '', 'basementExists': 'true', 'outdoor': {'garden': {'surface': ''}, 'terrace': {'exists': 'true'}}, 'specificities': {'SME': {'office': {'exists': ''}}}, 'wellnessEquipment': {'hasSwimmingPool': ''}, 'parking': {'parkingSpaceCount': {'indoor': '', 'outdoor': ''}}, 'condition': {'isNewlyBuilt': 'false'}}


Lets take a look at what's in our soup

In [7]:
# Locality
# <span class="classified__information--address-row"> 
# -->                                       Rue des Francais 1,41 
#                                <!----></span>


# Type of property (House/apartment)
# Located in link as indicated below, maybe start fitering before getting data, so we can split the poroperty types
# https://www.immoweb.be/en/classified/ [[apartment]] /for-sale/ans/4430/9777582?searchId=621c956edcf6b 

# Subtype of property (Bungalow, Chalet, Mansion, ...)
# I don't directly see this in the link, but there is an option to filter this on the main page so there should be a way to get this info


# Price
# <p class="classified__price"><span aria-hidden="true">
# -->                                   €204,900
#                                </span> <span class="sr-only">204900€</span></p>


# Type of sale (Exclusion of life sales)
# Not sure what is asked here specifically


# Number of rooms
# Is this number of bedrooms? casue if so:
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">
#                                    Bedrooms
#                                </th> <td class="classified-table__data">
# -->                                   3
#                                </td></tr>


#Area
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">
#                                    Living area
#                                </th> <td class="classified-table__data">
# -->                                   90
#
#                                    <span class="abbreviation"><span aria-hidden="true">
# -->                                           m²                                        </span> <span class="sr-only">
#                                            square meters
#                                        </span></span></td></tr>


# Fully equipped kitchen (Yes/No)
# Not sure but maybe
# <tr class="classified-table__row"><th scope="row" class="classified-table__header">Kitchen type</th> <td class="classified-table__data">
# -->                                   Installed
#                                </td></tr>


#Furnished (Yes/No)
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">Furnished</th> <td class="classified-table__data">
# -->                                   No
#                                </td></tr>


#Open fire (Yes/No)
# Not sure but maybe
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">Heating type</th> <td class="classified-table__data">
# -->                               Gas
#                            </td></tr>


#Terrace (Yes/No)
    #If yes: Area

    
#Garden (Yes/No)
    #If yes: Area

    
#Surface of the land


#Surface area of the plot of land


#Number of facades


#Swimming pool (Yes/No)


#State of the building (New, to be renovated, ...)
